In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
import warnings
warnings.filterwarnings("ignore")
import talib as ta

In [2]:
data = yf.download("BAJAJFINSV.NS",start ="2005-1-1",end = "2021-12-1", interval = '1D').dropna()

[*********************100%***********************]  1 of 1 completed


In [3]:
dd = data.copy()
dd

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2005-01-03,1149.000000,1159.800049,1125.199951,1143.349976,1121.443848,389100
2005-01-04,1157.000000,1157.000000,1125.550049,1130.699951,1109.036133,295200
2005-01-05,1120.000000,1134.000000,1070.000000,1089.099976,1068.233154,307600
2005-01-06,1099.900024,1125.000000,1035.050049,1072.750000,1052.196533,152900
2005-01-07,1096.650024,1108.900024,1050.099976,1096.699951,1075.687622,98100
...,...,...,...,...,...,...
2021-11-24,17432.000000,17647.000000,17140.400391,17408.699219,17408.699219,274024
2021-11-25,17349.000000,17522.000000,17225.250000,17368.849609,17368.849609,161620
2021-11-26,17200.000000,17237.199219,16610.000000,16682.550781,16682.550781,342486


In [4]:
dd['Low1'] = dd['Low'].shift(1)
dd['High1'] = dd['High'].shift(1)
dd['Open1'] = dd['Open'].shift(1)
dd['Close1'] = dd['Close'].shift(1)
dd['Volume1'] = dd['Volume'].shift(1)

In [5]:
def shape(data):
    
    Open = data['Open']
    Close = data['Close']
    High = data['High']
    Low = data['Low']
    Low1 = data['Low1']
    High1 = data['High1']
    Open1 = data['Open1']
    Close1 = data['Close1']
    
    if (Open < Close and Open > Low):
        if (High == Open):
                #b = 10
                return 10
        if (High > Close):
                #a = 1
                return 1
            
    if (Open < Close and Low == Open):
        if (Close > Open):
                c=3
                return c
        if (High == Close):
                d = 7
                return d
    
    if (Open > Close):
        if (Close == Low):
            if (High == Open):
                e = 4
                return e
            if (High > Open):
                f = 6
                return f
        if (Close > Low):
            if (High == Open):
                g= 11
                return g
            if (High > Open):
                h = 13
                return h
                
                
    if (Open == Close):
        if (Open == Low):
            if (High > Close):
                i = 5
                return i
                
        if (High == Close):
            if (Low < Open):
                j = 8
                return j
                
        if (Close > Low):
            if (Open < High):
                k = 12
                return k
                
        
    if (Open == Close):
        if (High == Low):
            if (High > (High1+Close1)/2):
                l = 2
                return l
                
            if (Low <  (High1+Close1)/2):
                m = 9
                return m
            
            if (Open1 == Close1):
                if (Low1 == High1):
                    n= 14
                    return n

In [6]:
def position(data):
    
    Open = data['Open']
    Close = data['Close']
    High = data['High']
    Low = data['Low']
    Low1 = data['Low1']
    High1 = data['High1']
    Open1 = data['Open1']
    Close1 = data['Close1']
    
    if High > High1:
        if Low < Low1:
            if Close > Close1:
                return 1
            if Close < Close1:
                return 2
        if Low > Low1:
            if Close > Close1:
                return 3
            if Close < Close1:
                return 4
    if High < High1:
        if Low < Low1:
            if Close > Close1:
                return 5
            if Close < Close1:
                return 6
        if Low > Low1:
            if Close > Close1:
                return 7
            if Close < Close1:
                return 8

In [7]:
dd['Shape'] = dd.apply(shape, axis='columns')
dd["C_Position"] = dd.apply(position ,axis = 'columns')


In [8]:
x = dd.Shape.unique()
x

array([13., 11.,  1.,  3.,  9., 12., 14.,  6., nan,  4.,  2.,  8.])

In [9]:
alpha= pd.get_dummies(dd.Shape)
beta = pd.get_dummies(dd.C_Position)

In [10]:
alpha

,1.0,2.0,3.0,4.0,6.0,8.0,9.0,11.0,12.0,13.0,14.0
Date,,,,,,,,,,,
2005-01-03,0,0,0,0,0,0,0,0,0,1,0
2005-01-04,0,0,0,0,0,0,0,1,0,0,0
2005-01-05,0,0,0,0,0,0,0,0,0,1,0
2005-01-06,0,0,0,0,0,0,0,0,0,1,0
2005-01-07,1,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
2021-11-24,0,0,0,0,0,0,0,0,0,1,0
2021-11-25,1,0,0,0,0,0,0,0,0,0,0
2021-11-26,0,0,0,0,0,0,0,0,0,1,0


In [11]:
ShapeEnc = pd.DataFrame()

for i in range(1,15):
    ShapeEnc[i] = 0
    
for i in x:
    if i > 0:
        ShapeEnc[i] = alpha[i]

ShapeEnc = ShapeEnc.astype(np.float)

ShapeEnc = ShapeEnc.fillna(0)

In [12]:
C_PositionEnc = beta.dropna()
C_PositionEnc

,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0
Date,,,,,,,,
2005-01-03,0,0,0,0,0,0,0,0
2005-01-04,0,0,0,0,0,0,0,1
2005-01-05,0,0,0,0,0,1,0,0
2005-01-06,0,0,0,0,0,1,0,0
2005-01-07,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...
2021-11-24,0,0,1,0,0,0,0,0
2021-11-25,0,0,0,0,0,0,0,1
2021-11-26,0,0,0,0,0,1,0,0


In [13]:
ShapeEnc

,1,2,3,4,5,6,7,8,9,10,11,12,13,14
Date,,,,,,,,,,,,,,
2005-01-03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2005-01-04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2005-01-05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2005-01-06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2005-01-07,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-11-24,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2021-11-25,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2021-11-26,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [14]:
Enc = ShapeEnc.join(C_PositionEnc, how = 'left', lsuffix='-Shape', rsuffix='-CandlePos')

In [15]:
def vwap(data):
    vp = data["Close"]*data["Volume"]+data["Close1"]*data["Volume1"]
    v = data["Volume"]+data["Volume1"]
    return(vp/v) 

dd["ema"] = ta.EMA(dd["Adj Close"],timeperiod = 21)
dd["vwap"] = dd.apply(vwap,axis = "columns")
dd["atr"] = ta.ATR(dd["High"],dd["Low"],dd["Adj Close"])
dd["returns1"] = dd["Close"].pct_change() 
dd["returns3"] = dd["Close"].pct_change(periods = 3)
dd["returns7"] = dd["Close"].pct_change(periods = 7)
dd['BBANDS_U'] = ta.BBANDS(dd["Close"], timeperiod=21)[0]
dd['BBANDS_M'] = ta.BBANDS(dd["Close"], timeperiod=21)[1]
dd['BBANDS_L'] = ta.BBANDS(dd["Close"], timeperiod=21)[2]
    
dd.drop(['Low1','High1','Open1','Close1','Volume1'],axis = 1,inplace = True)

In [16]:
dd["signal"] = 0
dd.loc[(dd.vwap < dd.ema)  & (dd.BBANDS_M > dd.BBANDS_L) & (dd.BBANDS_M < dd.BBANDS_U),"signal"] = 1
dd.loc[(dd.vwap > dd.ema)  & (dd.BBANDS_M < dd.BBANDS_L) & (dd.BBANDS_M > dd.BBANDS_U),"signal"] = -1       

In [17]:
df = dd[["vwap","ema","atr","returns1","returns3","returns7","BBANDS_U","BBANDS_M","BBANDS_L","signal"]]

In [18]:
df

,vwap,ema,atr,returns1,returns3,returns7,BBANDS_U,BBANDS_M,BBANDS_L,signal
Date,,,,,,,,,,
2005-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2005-01-04,1137.892885,NaN,NaN,-0.011064,NaN,NaN,NaN,NaN,NaN,0
2005-01-05,1109.472094,NaN,NaN,-0.036791,NaN,NaN,NaN,NaN,NaN,0
2005-01-06,1083.671287,NaN,NaN,-0.015012,-0.061748,NaN,NaN,NaN,NaN,0
2005-01-07,1082.110519,NaN,NaN,0.022326,-0.030070,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...
2021-11-24,17389.276920,17850.481919,585.227845,0.002338,-0.028600,-0.047341,18626.725847,17895.433222,17164.140596,1
2021-11-25,17393.915373,17806.697164,564.622285,-0.002289,0.017284,-0.042009,18620.125773,17861.228423,17102.331072,1
2021-11-26,16902.583109,17704.502038,578.495665,-0.039513,-0.039472,-0.083867,18658.649835,17781.666574,16904.683312,1


In [19]:
for i in range(0,df.shape[1] - 1):
    df.iloc[:,i] = df.iloc[:,i] / df.iloc[:,i].rolling(window = 21).mean()

In [20]:
df = df.join(Enc).dropna()
df

,vwap,ema,atr,returns1,returns3,returns7,BBANDS_U,BBANDS_M,BBANDS_L,signal,...,13,14,1.0-CandlePos,2.0-CandlePos,3.0-CandlePos,4.0-CandlePos,5.0-CandlePos,6.0-CandlePos,7.0-CandlePos,8.0-CandlePos
Date,,,,,,,,,,,,,,,,,,,,,
2005-03-02,0.993226,0.986727,0.880005,-11.448151,-795.165556,6.380204,0.987439,0.996920,1.007370,0,...,0.0,0.0,0,0,0,0,0,0,0,0
2005-03-03,1.018192,0.990959,0.931323,17.081138,19.361709,8.875818,0.993889,0.999905,1.006516,0,...,0.0,0.0,0,0,1,0,0,0,0,0
2005-03-04,1.039198,0.995734,0.937568,5.969691,12.410884,10.587449,1.001040,1.002100,1.003262,0,...,0.0,0.0,0,0,1,0,0,0,0,0
2005-03-07,1.045077,1.000601,0.959622,4.510703,10.285864,13.874014,1.007114,1.003698,0.999953,0,...,0.0,0.0,0,0,1,0,0,0,0,0
2005-03-08,1.039622,1.003137,0.950492,-16.598896,-0.107592,9.423094,1.009841,1.004758,0.999190,0,...,1.0,0.0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-11-24,0.969640,0.992927,1.009757,-0.750444,2.813877,3.193924,0.982586,0.989610,0.997347,1,...,1.0,0.0,0,0,1,0,0,0,0,0
2021-11-25,0.972612,0.991075,0.975535,1.295704,-2.155092,2.537104,0.983322,0.988215,0.993598,1,...,0.0,0.0,0,0,0,0,0,0,0,1
2021-11-26,0.948463,0.986312,0.999105,9.085429,4.235289,4.151315,0.986296,0.984493,0.982510,1,...,1.0,0.0,0,0,0,0,0,1,0,0
